In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import cv2
import os
import csv
import glob
import numpy as np


In [3]:
BASE_PATH = "/content/drive/MyDrive/PCB_DATASET"

TEMPLATE_DIR = f"{BASE_PATH}/PCB_USED"
IMAGE_DIR    = f"{BASE_PATH}/images"

OUT_ALIGN = f"{BASE_PATH}/outputs/aligned"
OUT_DIFF  = f"{BASE_PATH}/outputs/diff"
OUT_MASK  = f"{BASE_PATH}/outputs/mask"
ROI_ROOT  = f"{BASE_PATH}/data/rois"
CSV_PATH  = f"{ROI_ROOT}/annotations.csv"

for d in [OUT_ALIGN, OUT_DIFF, OUT_MASK, ROI_ROOT]:
    os.makedirs(d, exist_ok=True)


# 1. ECC ALIGNMENT

In [4]:
def align_images(template, test):
    temp_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    test_gray = cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)

    warp_mode = cv2.MOTION_EUCLIDEAN
    warp_matrix = np.eye(2, 3, dtype=np.float32)

    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 200, 1e-6)

    try:
        cc, warp_matrix = cv2.findTransformECC(temp_gray, test_gray, warp_matrix, warp_mode, criteria)
        aligned = cv2.warpAffine(test, warp_matrix, (template.shape[1], template.shape[0]),
                                 flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        return aligned
    except cv2.error as e:
        print("[ECC FAILED → Returning Test Image]", e)
        return test

# 2. PREPROCESS (Histogram Equalization Instead of Normalize)


In [5]:
def preprocess(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g = cv2.GaussianBlur(g, (5,5), 0)
    g = cv2.equalizeHist(g)
    return g

# 3. MORPHOLOGY CLEANER


In [6]:
def denoise_mask(mask, defect):
    mask = cv2.medianBlur(mask, 3)

    kernel_open = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_open, iterations=2)

    kernel_close = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel_close, iterations=1)

    return mask

# 4. CONNECTED COMPONENT FILTER


In [7]:
def cc_filter(mask, min_area=120):
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    cleaned = np.zeros_like(mask)

    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area >= min_area:
            cleaned[labels == i] = 255

    return cleaned

# MODULE-1 PIPELINE


In [8]:
def module1():
    defect_folders = os.listdir(IMAGE_DIR)

    for defect in defect_folders:
        folder_path = f"{IMAGE_DIR}/{defect}"
        if not os.path.isdir(folder_path): continue

        print(f"\n[MODULE-1] Processing defect type: {defect}")

        for fname in os.listdir(folder_path):
            if not fname.lower().endswith((".jpg",".png",".jpeg")): continue

            cid = fname.split("_")[0]
            template_candidates = glob.glob(f"{TEMPLATE_DIR}/{cid}.*")
            if not template_candidates:
                print("[NO TEMPLATE]:", fname)
                continue

            template = cv2.imread(template_candidates[0])
            test = cv2.imread(f"{folder_path}/{fname}")

            # 1) Align
            aligned = align_images(template, test)
            cv2.imwrite(f"{OUT_ALIGN}/{fname}", aligned)

            # 2) Preprocess
            temp_p = preprocess(template)
            test_p = preprocess(aligned)

            # 3) Subtract
            diff = cv2.absdiff(temp_p, test_p)
            cv2.imwrite(f"{OUT_DIFF}/{fname}", diff)

            # 4) Threshold
            _, otsu = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

            # 5) Morphology
            den = denoise_mask(otsu, defect)

            # 6) Component Filtering
            clean = cc_filter(den, min_area=120)

            cv2.imwrite(f"{OUT_MASK}/{fname}", clean)

            print(f"[OK] {fname} → MASK SAVED")

    print("\n✔ MODULE-1 COMPLETED ✔")


# Run Module-1
module1()


[MODULE-1] Processing defect type: Spur
[OK] 12_spur_08.jpg → MASK SAVED
[OK] 11_spur_10.jpg → MASK SAVED
[OK] 12_spur_03.jpg → MASK SAVED
[OK] 12_spur_01.jpg → MASK SAVED
[OK] 11_spur_09.jpg → MASK SAVED
[OK] 12_spur_02.jpg → MASK SAVED
[OK] 11_spur_04.jpg → MASK SAVED
[OK] 12_spur_04.jpg → MASK SAVED
[OK] 12_spur_10.jpg → MASK SAVED
[OK] 12_spur_07.jpg → MASK SAVED
[OK] 12_spur_05.jpg → MASK SAVED
[OK] 11_spur_07.jpg → MASK SAVED
[OK] 11_spur_06.jpg → MASK SAVED
[OK] 11_spur_05.jpg → MASK SAVED
[OK] 12_spur_09.jpg → MASK SAVED
[OK] 11_spur_08.jpg → MASK SAVED
[OK] 12_spur_06.jpg → MASK SAVED
[OK] 07_spur_03.jpg → MASK SAVED
[OK] 05_spur_02.jpg → MASK SAVED
[OK] 08_spur_06.jpg → MASK SAVED
[OK] 07_spur_02.jpg → MASK SAVED
[OK] 06_spur_07.jpg → MASK SAVED
[OK] 10_spur_01.jpg → MASK SAVED
[OK] 10_spur_03.jpg → MASK SAVED
[OK] 06_spur_09.jpg → MASK SAVED
[OK] 05_spur_09.jpg → MASK SAVED
[OK] 08_spur_07.jpg → MASK SAVED
[OK] 06_spur_04.jpg → MASK SAVED
[OK] 08_spur_05.jpg → MASK SAVED
[O

Module 2: Contour Detection and ROI Extraction

In [9]:
import os
import cv2
import csv
import numpy as np

BASE_PATH = "/content/drive/MyDrive/PCB_DATASET"

MASK_DIR   = f"{BASE_PATH}/outputs/mask"
ALIGN_DIR  = f"{BASE_PATH}/outputs/aligned"
ROI_ROOT   = f"{BASE_PATH}/data/rois"
CSV_PATH   = f"{ROI_ROOT}/annotations.csv"

os.makedirs(ROI_ROOT, exist_ok=True)


Initialize CSV

In [10]:
if not os.path.exists(CSV_PATH):
    with open(CSV_PATH,'w',newline='') as f:
        csv.writer(f).writerow(["filename","x","y","w","h","label"])


Extract ROIs for One Image

In [11]:
def extract_rois(mask, aligned, defect, base):
    roi_dir = os.path.join(ROI_ROOT, defect)
    os.makedirs(roi_dir, exist_ok=True)

    if defect.lower() == "spur":
        min_area = 300
        min_white = 50
    else:
        min_area = 80
        min_white = 20

    cnts,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    roi_count = 0
    rows = []

    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        if w*h < min_area: continue
        if cv2.countNonZero(mask[y:y+h,x:x+w]) < min_white: continue

        roi = aligned[y:y+h, x:x+w]
        name = f"{base}_roi_{roi_count:04d}.jpg"
        cv2.imwrite(f"{roi_dir}/{name}", roi)

        rows.append([name,x,y,w,h,defect])
        roi_count += 1

    with open(CSV_PATH,'a',newline='') as f:
        csv.writer(f).writerows(rows)

    return roi_count


In [12]:
def module2():
    files = os.listdir(OUT_MASK)
    total = 0

    for fname in files:
        if not fname.lower().endswith((".jpg",".png",".jpeg")): continue

        defect = fname.split("_")[1]
        base = fname.split(".")[0]

        mask = cv2.imread(f"{OUT_MASK}/{fname}",0)
        aligned = cv2.imread(f"{OUT_ALIGN}/{fname}")

        if mask is None or aligned is None:
            print("[MISSING]:", fname)
            continue

        count = extract_rois(mask, aligned, defect, base)
        total += count

        print(f"[OK] {fname} → {count} ROIs")

    print("\n✔ MODULE-2 COMPLETED — Total ROIs:", total)


In [13]:
module2()


[OK] 12_spur_08.jpg → 5 ROIs
[OK] 11_spur_10.jpg → 5 ROIs
[OK] 12_spur_03.jpg → 5 ROIs
[OK] 12_spur_01.jpg → 4 ROIs
[OK] 11_spur_09.jpg → 4 ROIs
[OK] 12_spur_02.jpg → 5 ROIs
[OK] 11_spur_04.jpg → 5 ROIs
[OK] 12_spur_04.jpg → 3 ROIs
[OK] 12_spur_10.jpg → 5 ROIs
[OK] 12_spur_07.jpg → 5 ROIs
[OK] 12_spur_05.jpg → 4 ROIs
[OK] 11_spur_07.jpg → 5 ROIs
[OK] 11_spur_06.jpg → 4 ROIs
[OK] 11_spur_05.jpg → 5 ROIs
[OK] 12_spur_09.jpg → 4 ROIs
[OK] 11_spur_08.jpg → 5 ROIs
[OK] 12_spur_06.jpg → 5 ROIs
[OK] 07_spur_03.jpg → 5 ROIs
[OK] 05_spur_02.jpg → 5 ROIs
[OK] 08_spur_06.jpg → 3 ROIs
[OK] 07_spur_02.jpg → 5 ROIs
[OK] 06_spur_07.jpg → 5 ROIs
[OK] 10_spur_01.jpg → 5 ROIs
[OK] 10_spur_03.jpg → 5 ROIs
[OK] 06_spur_09.jpg → 7 ROIs
[OK] 05_spur_09.jpg → 5 ROIs
[OK] 08_spur_07.jpg → 5 ROIs
[OK] 06_spur_04.jpg → 4 ROIs
[OK] 08_spur_05.jpg → 5 ROIs
[OK] 07_spur_09.jpg → 5 ROIs
[OK] 07_spur_05.jpg → 5 ROIs
[OK] 09_spur_10.jpg → 5 ROIs
[OK] 11_spur_02.jpg → 4 ROIs
[OK] 09_spur_06.jpg → 5 ROIs
[OK] 06_spur_0